# Download Package

In [1]:
!pip install simplet5

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 41.4 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6854 sha256=0b396688219f2371428a05f3438b3e4ed1ea0c50cc3b6b37a6c1338b74d1d803
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a 

# Import Library

In [2]:
# For data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# For model, tokenizer
from simplet5 import SimpleT5
import transformers
from transformers import AutoTokenizer, AutoConfig, T5ForConditionalGeneration

# For neccessary
import os
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load dataset

In [3]:
train_df = pd.read_csv("/kaggle/input/t5-skinchat-data-tokenizer-preparation/data_train.csv")
test_df = pd.read_csv("/kaggle/input/t5-skinchat-data-tokenizer-preparation/data_test.csv")
train_df.shape, test_df.shape

((66083, 2), (7346, 2))

In [4]:
train_df['source_text'] = train_df['source_text'].astype(str)
train_df['target_text'] = train_df['target_text'].astype(str)
test_df['source_text'] = test_df['source_text'].astype(str)
test_df['target_text'] = test_df['target_text'].astype(str)

# Load Tokenizer

In [5]:
MODEL_NAME = "t5-base"
MAX_VOCAB = 20000

In [6]:
new_tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/t5-skinchat-data-tokenizer-preparation/skinchat_tokenizer/")

# Load Model

In [7]:
config = AutoConfig.from_pretrained(MODEL_NAME)
config.vocab_size = new_tokenizer.vocab_size
model = T5ForConditionalGeneration(config)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

In [8]:
# instantiate
simplet5 = SimpleT5()

simplet5.model = model
simplet5.tokenizer = new_tokenizer

In [9]:
total_params = sum(p.numel() for p in simplet5.model.parameters())
print(f"Tổng số tham số của mô hình: {total_params}")
simplet5.model

Tổng số tham số của mô hình: 209531136


T5ForConditionalGeneration(
  (shared): Embedding(14716, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(14716, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

# Training model

In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [11]:
simplet5.train(train_df=train_df,
               eval_df=test_df,
               source_max_token_len=100,
               target_max_token_len=250,
               batch_size=16, 
               max_epochs=10, 
               use_gpu=True,
               precision=16
              )

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/core/lightning.py:2054: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
! (cd outputs; ls)

simplet5-epoch-0-train-loss-3.2921-val-loss-2.38
simplet5-epoch-1-train-loss-2.3029-val-loss-2.0333
simplet5-epoch-2-train-loss-2.04-val-loss-1.8654
simplet5-epoch-3-train-loss-1.8758-val-loss-1.7541
simplet5-epoch-4-train-loss-1.751-val-loss-1.6669
simplet5-epoch-5-train-loss-1.6474-val-loss-1.5931
simplet5-epoch-6-train-loss-1.556-val-loss-1.5334
simplet5-epoch-7-train-loss-1.4742-val-loss-1.4884
simplet5-epoch-8-train-loss-1.3991-val-loss-1.4626
simplet5-epoch-9-train-loss-1.3308-val-loss-1.4264
